# Google Trends Workbook

## Purpose
State the purpose of the notebook.

## Methodology
Obtain Google Trends data and merge with Walmart sales dataset.

## WIP - improvements

Notable TODOs:
- Get daily data and normalize with monthly;
- Merge with sales dataset;

## Results
Monthly data directly downloaded from Google Trends interface: https://trends.google.com/trends/?geo=US

## Suggested next steps
Complete TODOs and continuew with other preprocessing steps.

# Setup

## Library import
We import all the required Python libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

import datetime as datetime
from pytrends.request import TrendReq


# Data import
We retrieve all the required data for the analysis.

In [ ]:
calendar_data = pd.read_csv('walmart_sales_data/calendar.csv')
#print(calendar_data.dtypes)
print('\nMinimum date: ', calendar_data['date']. min())
print('Maximum date: ', calendar_data['date']. max())
print('\n',calendar_data.head())


Minimum date:  2011-01-29
Maximum date:  2016-06-19

          date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0        1  


In [ ]:
str(datetime.date.today())

'2022-04-25'

In [ ]:
# Change the date column to DateTimeIndex
calendar_data['date'] = pd.to_datetime(calendar_data['date'], format="%Y/%m/%d")
calendar_data.set_index(pd.DatetimeIndex(calendar_data['date']), inplace=True)
calendar_data.drop(columns=['date'], inplace=True)

In [ ]:
month_list = pd.date_range(start=calendar_data.index.min(), end=calendar_data.index.max(), freq='m').to_pydatetime().tolist()
month_list = [datetime.datetime(date.year, date.month, 1).date() for date in month_list]

In [ ]:
month_list

[datetime.date(2011, 1, 1),
 datetime.date(2011, 2, 1),
 datetime.date(2011, 3, 1),
 datetime.date(2011, 4, 1),
 datetime.date(2011, 5, 1),
 datetime.date(2011, 6, 1),
 datetime.date(2011, 7, 1),
 datetime.date(2011, 8, 1),
 datetime.date(2011, 9, 1),
 datetime.date(2011, 10, 1),
 datetime.date(2011, 11, 1),
 datetime.date(2011, 12, 1),
 datetime.date(2012, 1, 1),
 datetime.date(2012, 2, 1),
 datetime.date(2012, 3, 1),
 datetime.date(2012, 4, 1),
 datetime.date(2012, 5, 1),
 datetime.date(2012, 6, 1),
 datetime.date(2012, 7, 1),
 datetime.date(2012, 8, 1),
 datetime.date(2012, 9, 1),
 datetime.date(2012, 10, 1),
 datetime.date(2012, 11, 1),
 datetime.date(2012, 12, 1),
 datetime.date(2013, 1, 1),
 datetime.date(2013, 2, 1),
 datetime.date(2013, 3, 1),
 datetime.date(2013, 4, 1),
 datetime.date(2013, 5, 1),
 datetime.date(2013, 6, 1),
 datetime.date(2013, 7, 1),
 datetime.date(2013, 8, 1),
 datetime.date(2013, 9, 1),
 datetime.date(2013, 10, 1),
 datetime.date(2013, 11, 1),
 datetime.da

In [ ]:
month_list.append(datetime.date(2016, 6, 1))
month_list.append(datetime.date(2016, 7, 1))
month_list.append(datetime.date(2016, 8, 1))

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)
kw_list = ['walmart', 'hobbies', 'household', 'foods']
trends_walmart = pd.DataFrame()
trends_hobbies = pd.DataFrame()
trends_household = pd.DataFrame()
trends_foods = pd.DataFrame()

In [ ]:
for i in range(len(month_list) - 1):
    start = month_list[i]
    end = month_list[i+1]
    timeframe = str(start) + ' ' + str(end)
    pytrends.build_payload(
            kw_list=['walmart'], 
            geo='US', 
            timeframe=timeframe)
    df = pytrends.interest_by_region()
    df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
    df.loc[:, 'date'] = start
    trends_walmart = pd.concat([trends_walmart, df])
    
trends_walmart["state"] = trends_walmart.index
trends_walmart.set_index(["state", pd.DatetimeIndex(trends_walmart['date'])], inplace=True)
trends_walmart.drop(columns=['date'], inplace=True)

In [ ]:
trends_walmart

walmart
state      date               
California 2011-01-01       50
Texas      2011-01-01       79
Wisconsin  2011-01-01       68
California 2011-02-01       54
Texas      2011-02-01       82
...                        ...
           2016-06-01       59
Wisconsin  2016-06-01       52
California 2016-07-01       38
Texas      2016-07-01       65
Wisconsin  2016-07-01       51

[201 rows x 1 columns]

In [ ]:
for i in range(len(month_list) - 1):
    start = month_list[i]
    end = month_list[i+1]
    timeframe = str(start) + ' ' + str(end)
    pytrends.build_payload(
            kw_list=['hobbies'], 
            geo='US', 
            timeframe=timeframe)
    df = pytrends.interest_by_region()
    df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
    df.loc[:, 'date'] = start
    trends_hobbies = pd.concat([trends_hobbies, df])

In [ ]:
trends_hobbies["state"] = trends_hobbies.index
trends_hobbies.set_index(["state", pd.DatetimeIndex(trends_hobbies['date'])], inplace=True)
trends_hobbies.drop(columns=['date'], inplace=True)

In [ ]:
trends_hobbies

hobbies
state      date               
California 2011-01-01       60
Texas      2011-01-01       38
Wisconsin  2011-01-01       53
California 2011-02-01       51
Texas      2011-02-01       36
...                        ...
           2016-06-01       41
Wisconsin  2016-06-01       63
California 2016-07-01       47
Texas      2016-07-01       34
Wisconsin  2016-07-01       47

[201 rows x 1 columns]

In [ ]:
for i in range(len(month_list) - 1):
    start = month_list[i]
    end = month_list[i+1]
    timeframe = str(start) + ' ' + str(end)
    pytrends.build_payload(
            kw_list=['household'], 
            geo='US', 
            timeframe=timeframe)
    df = pytrends.interest_by_region()
    df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
    df.loc[:, 'date'] = start
    trends_household = pd.concat([trends_household, df])

In [ ]:
trends_household["state"] = trends_household.index
trends_household.set_index(["state", pd.DatetimeIndex(trends_household['date'])], inplace=True)
trends_household.drop(columns=['date'], inplace=True)

In [ ]:
trends_household

household
state      date                 
California 2011-01-01         50
Texas      2011-01-01         57
Wisconsin  2011-01-01         53
California 2011-02-01         53
Texas      2011-02-01         55
...                          ...
           2016-06-01         52
Wisconsin  2016-06-01         58
California 2016-07-01         66
Texas      2016-07-01         60
Wisconsin  2016-07-01         62

[201 rows x 1 columns]

In [ ]:
for i in range(len(month_list) - 1):
    start = month_list[i]
    end = month_list[i+1]
    timeframe = str(start) + ' ' + str(end)
    pytrends.build_payload(
            kw_list=['foods'], 
            geo='US', 
            timeframe=timeframe)
    df = pytrends.interest_by_region()
    df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
    df.loc[:, 'date'] = start
    trends_foods = pd.concat([trends_foods, df])

In [ ]:
trends_foods["state"] = trends_foods.index
trends_foods.set_index(["state", pd.DatetimeIndex(trends_foods['date'])], inplace=True)
trends_foods.drop(columns=['date'], inplace=True)

In [ ]:
trends_foods

foods
state      date             
California 2011-01-01     76
Texas      2011-01-01     69
Wisconsin  2011-01-01     94
California 2011-02-01     78
Texas      2011-02-01     66
...                      ...
           2016-06-01     53
Wisconsin  2016-06-01     66
California 2016-07-01     62
Texas      2016-07-01     58
Wisconsin  2016-07-01     69

[201 rows x 1 columns]

In [ ]:
trends_foods.index

MultiIndex([('California', '2011-01-01'),
            (     'Texas', '2011-01-01'),
            ( 'Wisconsin', '2011-01-01'),
            ('California', '2011-02-01'),
            (     'Texas', '2011-02-01'),
            ( 'Wisconsin', '2011-02-01'),
            ('California', '2011-03-01'),
            (     'Texas', '2011-03-01'),
            ( 'Wisconsin', '2011-03-01'),
            ('California', '2011-04-01'),
            ...
            ( 'Wisconsin', '2016-03-01'),
            ('California', '2016-04-01'),
            (     'Texas', '2016-04-01'),
            ( 'Wisconsin', '2016-04-01'),
            ('California', '2016-05-01'),
            (     'Texas', '2016-05-01'),
            ( 'Wisconsin', '2016-05-01'),
            ('California', '2016-06-01'),
            (     'Texas', '2016-06-01'),
            ( 'Wisconsin', '2016-06-01')],
           names=['state', 'date'], length=279)

In [ ]:
trends = trends_walmart.join([trends_hobbies, trends_household, trends_foods])

In [ ]:
trends.drop_duplicates(inplace=True)

In [ ]:
trends

walmart  hobbies  household  foods
state      date                                          
California 2011-01-01       50       60         50     76
Texas      2011-01-01       79       38         57     69
Wisconsin  2011-01-01       68       53         53     94
California 2011-02-01       54       51         53     78
Texas      2011-02-01       82       36         55     66
...                        ...      ...        ...    ...
           2016-06-01       59       41         52     53
Wisconsin  2016-06-01       52       63         58     66
California 2016-07-01       38       47         66     62
Texas      2016-07-01       65       34         60     58
Wisconsin  2016-07-01       51       47         62     69

[201 rows x 4 columns]

In [ ]:
trends.to_csv('google_trends_monthly.csv')

In [ ]:
trends

,foods,date
geoName,,
California,75,2011-01-01
Texas,69,2011-01-01
Wisconsin,94,2011-01-01
California,78,2011-02-01
Texas,66,2011-02-01
...,...,...
Texas,67,2016-04-01
Wisconsin,77,2016-04-01
California,60,2016-05-01


## Daily

In [ ]:
import time

daily_trends = pd.DataFrame()

for day in calendar_data.index:
    start = day.date()
    end = day + datetime.timedelta(days=1)
    timeframe = str(start) + ' ' + str(end)
    for kw in kw_list:
        pytrends.build_payload(
            kw_list=[kw], 
            geo='US', 
            timeframe=timeframe)
        df = pytrends.interest_by_region()
        df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
        df.loc[:, 'date'] = start
        time.sleep(1)
    trends_walmart = pd.concat([trends_walmart, df])

In [ ]:
import time

daily_trends = pd.DataFrame()

for kw in kw_list:
    kw_trends = pd.DataFrame()
    for day in calendar_data.index:
        start = day.date()
        end = (day + datetime.timedelta(days=1)).date()
        timeframe = str(start) + ' ' + str(end)
        pytrends.build_payload(
            kw_list=[kw], 
            geo='US', 
            timeframe=timeframe)
        df = pytrends.interest_by_region()
        df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
        df.loc[:, 'date'] = start
        df["state"] = df.index
        df.set_index(["state", pd.DatetimeIndex(df['date'])], inplace=True)
        df.drop(columns=['date'], inplace=True)
        kw_trends = pd.concat([kw_trends, df])
        time.sleep(1)
    kw_trends.to_csv(f'trends_{kw}_{datetime.date.today()}.csv')
    print(f"Done crawling for {kw} at {datetime.now}.")
    daily_trends.join(kw_trends, how="outer")

ResponseError: The request failed: Google returned a response with code 429.

In [ ]:
timeframe='2011-02-01 2011-03-01'
pytrends.build_payload(
    kw_list=[kw],
    geo='US',
    timeframe=timeframe)
df = pytrends.interest_over_time()

In [ ]:
df.index

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14', '2011-01-15', '2011-01-16',
               '2011-01-17', '2011-01-18', '2011-01-19', '2011-01-20',
               '2011-01-21', '2011-01-22', '2011-01-23', '2011-01-24',
               '2011-01-25', '2011-01-26', '2011-01-27', '2011-01-28',
               '2011-01-29', '2011-01-30', '2011-01-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [ ]:
daily_trends.shape

(0, 0)

In [ ]:
pytrends = TrendReq(hl='en-US', tz=480)
pytrends.build_payload(
    kw_list = ["walmart"], 
    year_start=2011, 
    month_start=1, 
    day_start=29, 
    hour_start=0, 
    year_end=2015, 
    month_end=6, 
    day_end=20, 
    geo='US-CA', 
    gprop='', 
    sleep=0)
pytrends.interest_over_time()

TypeError: 'months' is an invalid keyword argument for __new__()

In [ ]:
for i in range(len(month_list) - 1):
    start = month_list[i]
    end = month_list[i+1]
    timeframe = str(start) + ' ' + str(end)
    pytrends = TrendReq(hl='en-US', tz=480)
    for 
    pytrends.build_payload(
            kw_list=['foods'], 
            geo='US-CA', 
            timeframe=timeframe)
    df = pytrends.interest_over_time()
    df = df[df.index.isin(['California', 'Texas', 'Wisconsin'])]
    df.loc[:, 'date'] = start
    trends_foods = pd.concat([trends_foods, df])

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)
tx_daily_trends = pd.DataFrame()
for kw in kw_list:
    kw_trends = pd.DataFrame()
    for i in range(len(month_list) - 1):
        start = month_list[i]
        end = month_list[i+1] - datetime.timedelta(days=1)
        timeframe = str(start) + ' ' + str(end)
        pytrends.build_payload(
            kw_list=[kw], 
            geo='US-TX', 
            timeframe=timeframe)
        df = pytrends.interest_over_time()
        df.drop(columns=["isPartial"], inplace=True)
        kw_trends = pd.concat([kw_trends, df])
        time.sleep(1)
    kw_trends.to_csv(f'trends_tx_{kw}_{datetime.date.today()}.csv')
    print(f"Done crawling for {kw} at {datetime.datetime.now()}.")
    tx_daily_trends.join(kw_trends, how="outer")
tx_daily_trends.to_csv(f'trends_tx_{datetime.date.today()}.csv')

Done crawling for walmart at 2022-04-25 10:48:27.970597.
Done crawling for hobbies at 2022-04-25 10:50:13.053128.
Done crawling for household at 2022-04-25 10:51:57.247080.
Done crawling for foods at 2022-04-25 10:53:39.697249.


In [ ]:
trends

'2022-04-25 09:51:37.563641'

# Data processing
Put here the core of the notebook. Feel free di further split this section into subsections.

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2

In [ ]:
new_df = pd.DataFrame()

In [ ]:
new_df

""
